In [209]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\richt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\richt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\richt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [210]:
# Load the df_main dataframe from the provided pickle file
df_main_images = pd.read_pickle("df_main_images.pkl")
df_main_images.shape

(131248, 58)

In [211]:
df_main_images.tail()

,body_id,assembly_id,name,volume,area,material_category,properties_vertex_count,properties_edge_count,properties_face_count,properties_loop_count,properties_shell_count,properties_body_count,properties_area,properties_volume,properties_likes_count,properties_comments_count,properties_views_count,properties_category_Mechanical Engineering,properties_category_Product Design,properties_category_Miscellaneous,properties_category_Electronics,properties_category_Machine design,properties_category_Furniture + Household,properties_category_Tools,properties_category_Automotive,properties_category_Design,properties_category_Toys,properties_industry_Other Industries,"properties_industry_Architecture, Engineering & Construction",properties_industry_Product Design & Manufacturing,properties_industry_Civil Infrastructure,properties_industry_Media & Entertainment,components_name,body_file_size,CYLINDRICAL_SURFACE,SPHERICAL_SURFACE,PLANE,assembly_complexity,assembly_prediction_1,assembly_probability_1,assembly_prediction_2,assembly_probability_2,assembly_prediction_3,assembly_probability_3,assembly_prediction_4,assembly_probability_4,assembly_prediction_5,assembly_probability_5,prediction_1,probability_1,prediction_2,probability_2,prediction_3,probability_3,prediction_4,probability_4,prediction_5,probability_5
131243,f95a09b6-0613-11ec-b131-0226f5980261,99852_1dadcef4,NaN,0.116680,1.729151,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,manubrio,17938,4,0,6,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loupe,29.129223,spotlight,24.932516,whistle,19.307722,binoculars,16.574205,switch,10.056341
131244,f95a5828-0613-11ec-ab9b-0226f5980261,99852_1dadcef4,NaN,0.002853,0.129364,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo doble,6562,1,1,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,pedestal,22.592855,loudspeaker,22.528826,ashcan,20.789967,soap_dispenser,17.077143,radio,17.011213
131245,f95a7efe-0613-11ec-bc18-0226f5980261,99852_1dadcef4,NaN,0.000024,0.016467,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo doble,7424,2,0,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loupe,84.049187,face_powder,12.793128,puck,1.281135,toilet_seat,1.050157,buckle,0.826395
131246,f95acd26-0613-11ec-a896-0226f5980261,99852_1dadcef4,NaN,0.002356,0.109956,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo sencillo,5677,1,0,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loudspeaker,44.669567,radio,18.949003,soap_dispenser,18.021702,can_opener,10.712275,ashcan,7.647449
131247,f95af41c-0613-11ec-8208-0226f5980261,99852_1dadcef4,NaN,0.001571,0.094248,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo sencillo,5678,1,0,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,face_powder,66.571045,puck,23.176563,loupe,5.981135,frying_pan,3.050615,washbasin,1.220639


In [212]:
# Load the df_main dataframe from the provided pickle file
df_main_images_test = pd.read_pickle("df_test_main_images.pkl")
df_main_images_test.shape

(155, 58)

In [213]:
df_main_images_test.tail()

,body_id,assembly_id,name,volume,area,material_category,properties_vertex_count,properties_edge_count,properties_face_count,properties_loop_count,properties_shell_count,properties_body_count,properties_area,properties_volume,properties_likes_count,properties_comments_count,properties_views_count,properties_category_Mechanical Engineering,properties_category_Product Design,properties_category_Miscellaneous,properties_category_Electronics,properties_category_Machine design,properties_category_Furniture + Household,properties_category_Tools,properties_category_Automotive,properties_category_Design,properties_category_Toys,properties_industry_Other Industries,"properties_industry_Architecture, Engineering & Construction",properties_industry_Product Design & Manufacturing,properties_industry_Civil Infrastructure,properties_industry_Media & Entertainment,components_name,body_file_size,CYLINDRICAL_SURFACE,SPHERICAL_SURFACE,PLANE,assembly_complexity,assembly_prediction_1,assembly_probability_1,assembly_prediction_2,assembly_probability_2,assembly_prediction_3,assembly_probability_3,assembly_prediction_4,assembly_probability_4,assembly_prediction_5,assembly_probability_5,prediction_1,probability_1,prediction_2,probability_2,prediction_3,probability_3,prediction_4,probability_4,prediction_5,probability_5
150,3b4ce74c-05cc-11ec-a9ce-064a63348d37,74576_73ac0093,Pata_1,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Patas,9071,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,30.820402,pole,24.868954,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613
151,3b4d0e4c-05cc-11ec-9c6d-064a63348d37,74576_73ac0093,Pata_4,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Patas,9149,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,30.820402,pole,24.868954,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613
152,3b4d355a-05cc-11ec-9f42-064a63348d37,74576_73ac0093,Pata_3,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Patas,9123,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,30.820402,pole,24.868954,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613
153,3b4d8388-05cc-11ec-acde-064a63348d37,74576_73ac0093,Pata_2,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Patas,9097,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,30.820402,pole,24.868954,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613
154,3b4daaa2-05cc-11ec-bc9f-064a63348d37,74576_73ac0093,Body1,33436.374783,27786.304206,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,Tabla,23759,8,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,hard_disc,30.939892,envelope,21.721859,scale,19.142361,mortarboard,17.675533,binder,10.520355


In [214]:
for i in range(1, 6):
    df_main_images[f'probability_{i}'] = df_main_images[f'probability_{i}'].replace('', np.nan).astype(float)

In [215]:
for i in range(1, 6):
    df_main_images_test[f'probability_{i}'] = df_main_images_test[f'probability_{i}'].replace('', np.nan).astype(float)

In [216]:
for i in range(1, 6):
    non_numeric = df_main_images[f'probability_{i}'][~df_main_images[f'probability_{i}'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()]
    if not non_numeric.empty:
        print(f"Non-numeric values in probability_{i}: {non_numeric.tolist()}")


Non-numeric values in probability_1: [nan, nan, nan]
Non-numeric values in probability_2: [nan, nan, nan]
Non-numeric values in probability_3: [nan, nan, nan]
Non-numeric values in probability_4: [nan, nan, nan]
Non-numeric values in probability_5: [nan, nan, nan]


In [217]:
for i in range(1, 6):
    non_numeric = df_main_images_test[f'probability_{i}'][~df_main_images_test[f'probability_{i}'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()]
    if not non_numeric.empty:
        print(f"Non-numeric values in probability_{i}: {non_numeric.tolist()}")


In [218]:
def process_prediction(df, prediction_col, probability_col):
    # Convert probabilities from 0-100 scale to 0-1 scale
    df[probability_col] = df[probability_col] / 100.0
    
    # Calculate chi-square residuals
    contingency = pd.crosstab(df[prediction_col], df['material_category'])
    chi2, p, _, expected = chi2_contingency(contingency)
    residuals = (contingency - expected) / np.sqrt(expected)
    max_residuals = residuals.abs().max(axis=1)

    # Obtain the overall counts of each prediction
    prediction_counts = df[prediction_col].value_counts()

    # Combine residuals with counts to get a composite score
    composite_scores = max_residuals * prediction_counts

    # Sort by this composite score
    top_predictions = composite_scores.sort_values(ascending=False).head(10).index.tolist()

    # Manually add "screw" and "coil" if they are not already in the list
    for prediction in ["screw", "coil"]:
        if prediction not in top_predictions:
            top_predictions.append(prediction)

    # Create new columns in the dataframe
    for prediction in top_predictions:
        column_name = f"{prediction_col}_{prediction}"
        df[column_name] = np.where(df[prediction_col] == prediction, df[probability_col], 0)

    return df, top_predictions




In [219]:
def apply_top_predictions(df, top_predictions, prediction_col, probability_col):
    # Convert probabilities from 0-100 scale to 0-1 scale
    df[probability_col] = df[probability_col] / 100.0

    # Create new columns in the dataframe based on top_predictions
    for prediction in top_predictions:
        column_name = f"{prediction_col}_{prediction}"
        df[column_name] = np.where(df[prediction_col] == prediction, df[probability_col], 0)

    return df


In [220]:
# Process for prediction_1 and probability_1 for df_main_images
df_main_images, top_predictions_1 = process_prediction(df_main_images, 'prediction_1', 'probability_1')

# Process for prediction_2 and probability_2 for df_main_images
df_main_images, top_predictions_2 = process_prediction(df_main_images, 'prediction_2', 'probability_2')

# Display the updated dataframe
print(df_main_images.tail())

                                     body_id     assembly_id name    volume  \
131243  f95a09b6-0613-11ec-b131-0226f5980261  99852_1dadcef4  NaN  0.116680   
131244  f95a5828-0613-11ec-ab9b-0226f5980261  99852_1dadcef4  NaN  0.002853   
131245  f95a7efe-0613-11ec-bc18-0226f5980261  99852_1dadcef4  NaN  0.000024   
131246  f95acd26-0613-11ec-a896-0226f5980261  99852_1dadcef4  NaN  0.002356   
131247  f95af41c-0613-11ec-8208-0226f5980261  99852_1dadcef4  NaN  0.001571   

            area    material_category  properties_vertex_count  \
131243  1.729151  Metal_Ferrous_Steel                     3511   
131244  0.129364  Metal_Ferrous_Steel                     3511   
131245  0.016467  Metal_Ferrous_Steel                     3511   
131246  0.109956  Metal_Ferrous_Steel                     3511   
131247  0.094248  Metal_Ferrous_Steel                     3511   

        properties_edge_count  properties_face_count  properties_loop_count  \
131243                   5189                   1

In [221]:
# Apply these top predictions to df_main_images_test
df_main_images_test = apply_top_predictions(df_main_images_test, top_predictions_1, 'prediction_1', 'probability_1')

# Apply these top predictions to df_main_images_test
df_main_images_test = apply_top_predictions(df_main_images_test, top_predictions_2, 'prediction_2', 'probability_2')

# Display the updated dataframe
print(df_main_images_test.tail())

                                  body_id     assembly_id    name  \
150  3b4ce74c-05cc-11ec-a9ce-064a63348d37  74576_73ac0093  Pata_1   
151  3b4d0e4c-05cc-11ec-9c6d-064a63348d37  74576_73ac0093  Pata_4   
152  3b4d355a-05cc-11ec-9f42-064a63348d37  74576_73ac0093  Pata_3   
153  3b4d8388-05cc-11ec-acde-064a63348d37  74576_73ac0093  Pata_2   
154  3b4daaa2-05cc-11ec-bc9f-064a63348d37  74576_73ac0093   Body1   

           volume          area material_category  properties_vertex_count  \
150   2064.512000   1677.212800              None                      120   
151   2064.512000   1677.212800              None                      120   
152   2064.512000   1677.212800              None                      120   
153   2064.512000   1677.212800              None                      120   
154  33436.374783  27786.304206              None                      120   

     properties_edge_count  properties_face_count  properties_loop_count  \
150                    184              

In [222]:
df_main_images["components_name"].unique()

array(['root', 'Hinge', 'Base', ..., 'sujetador', 'tornillo doble',
       'tornillo sencillo'], dtype=object)

In [223]:
df_main_images_test["components_name"].unique()

array(['Component5', 'root', 'Component4', 'Component9', 'Component7',
       'Component8', 'Component10', 'Component3', 'Component6',
       'Component1', 'Component2', 'Component13', 'Component14',
       'Component11', 'Component12', 'ADORNO 1', 'BAJO LUZ', 'BASE',
       'SOPORTE', 'BARRA LARGA', 'TAPADERA BASE', 'TULIPA', 'SOBRE LUZ',
       'LATERAL', 'CENTRAL', 'TAPADERA', 'Component15', 'Component18',
       'Component19', 'Component21', 'Component17', 'Component20',
       'Component16', 'Pistón', 'Asiento', 'Base Silla', 'Descansabrazos',
       'Base Descansabrazos', 'Rueda Inventor v1 (1)',
       'Rueda Inventor v1 (2)', 'Rueda Inventor v1 (3)',
       'Rueda Inventor v1 (4)', 'Base Asiento', 'Rueda Inventor v1',
       'rubber pad', 'vertical axis', 'link 1', 'joint 2a',
       'head mount screw 2', 'bulb socket', 'joint 2b', 'link 2',
       'thumb screw 1', 'joint 1 fastener', 'joint 1', 'head mount',
       'joint 3 fastener', 'mounting sleeve', 'joint 2 fastener',
   

In [224]:
# Assuming df_main_images is your DataFrame

# Step 1: Convert to lowercase
df_main_images['name'] = df_main_images['name'].str.lower()
df_main_images['components_name'] = df_main_images['components_name'].str.lower()

# Step 2: Remove digits and non-alphabetic characters
df_main_images['name'] = df_main_images['name'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x)))
df_main_images['components_name'] = df_main_images['components_name'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x)))

# Step 3: Remove non-ASCII characters
df_main_images['name'] = df_main_images['name'].apply(lambda x: ''.join([char for char in x if ord(char) < 128]))
df_main_images['components_name'] = df_main_images['components_name'].apply(lambda x: ''.join([char for char in x if ord(char) < 128]))

# Step 4: Lemmatize the words
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    return ' '.join(tokens)

df_main_images['name'] = df_main_images['name'].apply(lemmatize_text)
df_main_images['components_name'] = df_main_images['components_name'].apply(lemmatize_text)


In [225]:
# Assuming df_main_images is your DataFrame

# Step 1: Convert to lowercase
df_main_images_test['name'] = df_main_images_test['name'].str.lower()
df_main_images_test['components_name'] = df_main_images_test['components_name'].str.lower()

# Step 2: Remove digits and non-alphabetic characters
df_main_images_test['name'] = df_main_images_test['name'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x)))
df_main_images_test['components_name'] = df_main_images_test['components_name'].apply(lambda x: re.sub(r'[^a-z\s]', '', str(x)))

# Step 3: Remove non-ASCII characters
df_main_images_test['name'] = df_main_images_test['name'].apply(lambda x: ''.join([char for char in x if ord(char) < 128]))
df_main_images_test['components_name'] = df_main_images_test['components_name'].apply(lambda x: ''.join([char for char in x if ord(char) < 128]))

# Step 4: Lemmatize the words
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords.words('english')]
    return ' '.join(tokens)

df_main_images_test['name'] = df_main_images_test['name'].apply(lemmatize_text)
df_main_images_test['components_name'] = df_main_images_test['components_name'].apply(lemmatize_text)


In [226]:
df_main_images['name'].nunique()

4355

In [227]:
df_main_images_test['name'].nunique()

61

In [228]:
df_main_images['components_name'].nunique()

11212

In [229]:
df_main_images_test['components_name'].nunique()

43

In [230]:

# 1. Text Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X_name = vectorizer.fit_transform(df_main_images['name'])
X_comp_name = vectorizer.fit_transform(df_main_images['components_name'])

# 2. Dimensionality Reduction
pca = PCA(n_components=25)
X_name_pca = pca.fit_transform(X_name.toarray())
X_comp_name_pca = pca.fit_transform(X_comp_name.toarray())

# 3. Clustering
# For simplicity, I'll use KMeans, assuming 100 clusters for `name` and 200 for `component_name`.
# You might want to optimize these numbers based on domain knowledge or experimentation.
kmeans_name = KMeans(n_clusters=2).fit(X_name_pca)
kmeans_comp_name = KMeans(n_clusters=3).fit(X_comp_name_pca)

# 4. Assign Cluster IDs
df_main_images['name_cluster'] = kmeans_name.labels_
df_main_images['comp_name_cluster'] = kmeans_comp_name.labels_

# Optional: Convert cluster IDs to category type for efficiency
df_main_images['name_cluster'] = df_main_images['name_cluster'].astype('category')
df_main_images['comp_name_cluster'] = df_main_images['comp_name_cluster'].astype('category')


C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [231]:

# 1. Text Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X_name = vectorizer.fit_transform(df_main_images_test['name'])
X_comp_name = vectorizer.fit_transform(df_main_images_test['components_name'])

# 2. Dimensionality Reduction
pca = PCA(n_components=25)
X_name_pca = pca.fit_transform(X_name.toarray())
X_comp_name_pca = pca.fit_transform(X_comp_name.toarray())

# 3. Clustering
# For simplicity, I'll use KMeans, assuming 100 clusters for `name` and 200 for `component_name`.
# You might want to optimize these numbers based on domain knowledge or experimentation.
kmeans_name = KMeans(n_clusters=2).fit(X_name_pca)
kmeans_comp_name = KMeans(n_clusters=3).fit(X_comp_name_pca)

# 4. Assign Cluster IDs
df_main_images_test['name_cluster'] = kmeans_name.labels_
df_main_images_test['comp_name_cluster'] = kmeans_comp_name.labels_

# Optional: Convert cluster IDs to category type for efficiency
df_main_images_test['name_cluster'] = df_main_images_test['name_cluster'].astype('category')
df_main_images_test['comp_name_cluster'] = df_main_images_test['comp_name_cluster'].astype('category')


C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\richt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [234]:
df_main_images.tail()

,body_id,assembly_id,name,volume,area,material_category,properties_vertex_count,properties_edge_count,properties_face_count,properties_loop_count,properties_shell_count,properties_body_count,properties_area,properties_volume,properties_likes_count,properties_comments_count,properties_views_count,properties_category_Mechanical Engineering,properties_category_Product Design,properties_category_Miscellaneous,properties_category_Electronics,properties_category_Machine design,properties_category_Furniture + Household,properties_category_Tools,properties_category_Automotive,properties_category_Design,properties_category_Toys,properties_industry_Other Industries,"properties_industry_Architecture, Engineering & Construction",properties_industry_Product Design & Manufacturing,properties_industry_Civil Infrastructure,properties_industry_Media & Entertainment,components_name,body_file_size,CYLINDRICAL_SURFACE,SPHERICAL_SURFACE,PLANE,assembly_complexity,assembly_prediction_1,assembly_probability_1,assembly_prediction_2,assembly_probability_2,assembly_prediction_3,assembly_probability_3,assembly_prediction_4,assembly_probability_4,assembly_prediction_5,assembly_probability_5,prediction_1,probability_1,prediction_2,probability_2,prediction_3,probability_3,prediction_4,probability_4,prediction_5,probability_5,prediction_1_balance_beam,prediction_1_hook,prediction_1_letter_opener,prediction_1_mortarboard,prediction_1_radio,prediction_1_face_powder,prediction_1_ping-pong_ball,prediction_1_binder,prediction_1_loupe,prediction_1_broom,prediction_1_screw,prediction_1_coil,prediction_2_letter_opener,prediction_2_hook,prediction_2_radio,prediction_2_binder,prediction_2_switch,prediction_2_loupe,prediction_2_envelope,prediction_2_loudspeaker,prediction_2_maraca,prediction_2_academic_gown,prediction_2_screw,prediction_2_coil,name_cluster,comp_name_cluster
131243,f95a09b6-0613-11ec-b131-0226f5980261,99852_1dadcef4,nan,0.116680,1.729151,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,manubrio,17938,4,0,6,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loupe,0.291292,spotlight,0.249325,whistle,19.307722,binoculars,16.574205,switch,10.056341,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.291292,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0,1
131244,f95a5828-0613-11ec-ab9b-0226f5980261,99852_1dadcef4,nan,0.002853,0.129364,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo doble,6562,1,1,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,pedestal,0.225929,loudspeaker,0.225288,ashcan,20.789967,soap_dispenser,17.077143,radio,17.011213,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.225288,0.0,0.0,0.0,0.0,0,1
131245,f95a7efe-0613-11ec-bc18-0226f5980261,99852_1dadcef4,nan,0.000024,0.016467,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo doble,7424,2,0,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loupe,0.840492,face_powder,0.127931,puck,1.281135,toilet_seat,1.050157,buckle,0.826395,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.840492,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0,1
131246,f95acd26-0613-11ec-a896-0226f5980261,99852_1dadcef4,nan,0.002356,0.109956,Metal_Ferrous_Steel,3511,5189,1902,2420,60,54,2121.693737,299.830188,0,0,161,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,tornillo sencillo,5677,1,0,3,34651737258,knee_pad,36.787369,whistle,18.592934,spotlight,17.50428,loupe,15.573485,cassette_player,11.541929,loudspeaker,0.446696,radio,0.189490,soap_dispenser,18.021702,can_opener,10.712275,ashcan,7.647449,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.18949,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0,

In [235]:
df_main_images.shape

(131248, 84)

In [236]:
df_main_images_test.tail()

,body_id,assembly_id,name,volume,area,material_category,properties_vertex_count,properties_edge_count,properties_face_count,properties_loop_count,properties_shell_count,properties_body_count,properties_area,properties_volume,properties_likes_count,properties_comments_count,properties_views_count,properties_category_Mechanical Engineering,properties_category_Product Design,properties_category_Miscellaneous,properties_category_Electronics,properties_category_Machine design,properties_category_Furniture + Household,properties_category_Tools,properties_category_Automotive,properties_category_Design,properties_category_Toys,properties_industry_Other Industries,"properties_industry_Architecture, Engineering & Construction",properties_industry_Product Design & Manufacturing,properties_industry_Civil Infrastructure,properties_industry_Media & Entertainment,components_name,body_file_size,CYLINDRICAL_SURFACE,SPHERICAL_SURFACE,PLANE,assembly_complexity,assembly_prediction_1,assembly_probability_1,assembly_prediction_2,assembly_probability_2,assembly_prediction_3,assembly_probability_3,assembly_prediction_4,assembly_probability_4,assembly_prediction_5,assembly_probability_5,prediction_1,probability_1,prediction_2,probability_2,prediction_3,probability_3,prediction_4,probability_4,prediction_5,probability_5,prediction_1_balance_beam,prediction_1_hook,prediction_1_letter_opener,prediction_1_mortarboard,prediction_1_radio,prediction_1_face_powder,prediction_1_ping-pong_ball,prediction_1_binder,prediction_1_loupe,prediction_1_broom,prediction_1_screw,prediction_1_coil,prediction_2_letter_opener,prediction_2_hook,prediction_2_radio,prediction_2_binder,prediction_2_switch,prediction_2_loupe,prediction_2_envelope,prediction_2_loudspeaker,prediction_2_maraca,prediction_2_academic_gown,prediction_2_screw,prediction_2_coil,name_cluster,comp_name_cluster
150,3b4ce74c-05cc-11ec-a9ce-064a63348d37,74576_73ac0093,pata,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,patas,9071,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,0.308204,pole,0.248690,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1,2
151,3b4d0e4c-05cc-11ec-9c6d-064a63348d37,74576_73ac0093,pata,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,patas,9149,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,0.308204,pole,0.248690,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1,2
152,3b4d355a-05cc-11ec-9f42-064a63348d37,74576_73ac0093,pata,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,patas,9123,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,0.308204,pole,0.248690,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1,2
153,3b4d8388-05cc-11ec-acde-064a63348d37,74576_73ac0093,pata,2064.512000,1677.212800,None,120,184,90,90,13,13,47899.853006,58194.002687,0,0,397,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,patas,9097,0,0,7,1987200,dining_table,94.939163,folding_chair,2.509808,desk,1.274445,potter's_wheel,0.681814,mortarboard,0.594778,sliding_door,0.308204,pole,0.248690,table_lamp,17.045835,guillotine,15.840197,monitor,11.424613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1,2
154,3b4daaa2-05cc-11ec-bc9f-064a63348d37,74576_73ac0093,body,33436.374783,27786.304206,None,120,184,90,90,13,13,4789

In [237]:
df_main_images_test.shape

(155, 84)

In [232]:
df_main_images.to_pickle('df_main_fe2.pkl')

In [233]:
df_main_images_test.to_pickle('df_test_main_fe2.pkl')